<a href="https://colab.research.google.com/github/ganbozza/test/blob/main/Mega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://colab.research.google.com/drive/1vIw-njmyXFclK2dhbB3NjI5Hj1HWgTFl


!sudo apt-get update && sudo apt-get install fuse3
import os

if not os.path.exists("/usr/bin/rclone"):
  print("[!] Installing rclone")
  !sudo -v ; curl https://www.esolat.com/swarmui/rclone_install.sh | sudo bash
  #!sudo -v ; curl https://rclone.org/install.sh | sudo bash

print("----------------------------------------")
print("[!] SUCCESS : Proceed to next step.")
print("----------------------------------------")


In [ ]:
# mega drive #1, that (will) contains swarmui installation
import os

if not os.path.exists("/usr/bin/rclone"):
  raise Exception("[x] ERROR : rclone not installed.")

if not os.path.exists("/root/.config/rclone"):
  !mkdir /root/.config/rclone/
!sudo wget https://www.esolat.com/swarmui/rclone.conf -O /root/.config/rclone/rclone.conf

if not os.path.exists("/content/mega-main"):
  !mkdir /content/mega-main/

!nohup rclone --vfs-cache-mode full mount mega-main: ./mega-main &

if not os.path.exists("/content/mega-drive2"):
  !mkdir /content/mega-drive2/

!nohup rclone --vfs-cache-mode full mount mega-drive2: ./mega-drive2 &

print("----------------------------------------")
print("[!] SUCCESS : Proceed to next step.")
print("----------------------------------------")

In [ ]:
SWARMPATH = '/content/mega-main'
if os.path.exists("/content/mega-main"):
  # Colab breaks venvs, and doesn't save anything valid to drive, so just screw it do a global install of pip reqs if we used a comfy backend in the drive
  !if [[ -f "$SWARMPATH/SwarmUI/dlbackend/ComfyUI/requirements.txt" ]]; then rm -rf $SWARMPATH/SwarmUI/dlbackend/ComfyUI/venv/; pip install -r $SWARMPATH/SwarmUI/dlbackend/ComfyUI/requirements.txt; fi
else:
  raise Exception("[x] $SWARMPATH drive not mounted?")

print("----------------------------------------")
print("[!] SUCCESS : Proceed to next step.")
print("----------------------------------------")


In [ ]:
# Install dotnet dependencies
!wget https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!chmod +x dotnet-install.sh
!./dotnet-install.sh --channel 8.0

# Install Clouldflared
!wget https://github.com/cloudflare/cloudflared/releases/download/2024.8.2/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# use the chosen path
import os
os.environ['SWARMPATH'] = SWARMPATH
%cd $SWARMPATH

# Colab breaks venv, so, tell swarm to not make a venv
os.environ['SWARM_NO_VENV'] = 'true'

# Download SwarmUI
url = "https://github.com/mcmonkeyprojects/SwarmUI"
!git clone $url

print("----------------------------------------")
print("[!] SUCCESS : Proceed to next step.")
print("----------------------------------------")

In [ ]:
# Alright, launch it! Watch the output for a Cloudflare URL
%cd $SWARMPATH/SwarmUI/

# (Colab stupid-proofing: aggressive git ultraforce pull)
!git fetch
!git reset --hard origin/master
!git pull --autostash
# (Colab stupid-proofing: drive wonks the perms so this needs an aggressive rebuild)
!rm -rf ./src/bin/live_release

!bash ./launch-linux.sh --launch_mode none --cloudflared-path cloudflared